In [ ]:
# Install necessary libraries
!pip install medmnist numpy pandas tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Paths
drive_output_dir = '/content/drive/MyDrive/pathmnist/'  # Change this to your desired Drive folder

In [ ]:
from medmnist import INFO, PathMNIST
import os
import numpy as np

# Download PathMNIST dataset
data_path = "/root/.medmnist"  # Default path used by medmnist
PathMNIST(root=data_path, split="train", download=True, size=224)

# MedMNIST info for PathMNIST
dataset_info = INFO['pathmnist']
label_text_values = list(dataset_info['label'].values())  # Get all values from label_text dictionary
print (label_text_values)

data = np.load("/root/.medmnist/pathmnist_224.npz")
images = data["train_images"]  # Training images
labels = data["train_labels"]  # Corresponding labels

# Manual label mapping
label_mapping = {
    0: "adipose",
    1: "background",
    2: "debris",
    3: "lymphocytes",
    4: "mucus",
    5: "smooth muscle",
    6: "normal colon mucosa",
    7: "cancer-associated stroma",
    8: "colorectal adenocarcinoma epithelium"
}

In [ ]:
# Import tqdm
from tqdm import tqdm
# Import the Image module from Pillow
from PIL import Image

# Create the output directory on Drive
image_dir = os.path.join(drive_output_dir, 'images')
os.makedirs(image_dir, exist_ok=True)

# Save images
metadata = []
for idx, (img, label) in enumerate(tqdm(zip(images, labels), desc="Processing images", total=len(labels))):

    # Convert the numpy array to an image (RGB format)
    pil_img = Image.fromarray(img)

    # Get the disease name using the label mapping and format the label description
    disease_name = label_mapping[label[0]]
    label_description = f"a histopathological image of an area with {disease_name}"

    # Create a subdirectory for the label if it doesn't exist
    # label_dir = os.path.join(image_dir, disease_name)
    label_dir = os.path.join(image_dir, label_description)
    os.makedirs(label_dir, exist_ok=True)

    # Save image as PNG under the label directory
    file_name = f'image_{idx}.png'
    image_path = os.path.join(label_dir, file_name)
    pil_img.save(image_path)

    # Append to metadata
    metadata.append([file_name, label_description])

print(f"Dataset prepared and saved to {drive_output_dir}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json

# Paths
base_dir = '/content/drive/MyDrive/pathmnist/images'
output_file = '/content/drive/MyDrive/pathmnist/images/metadata.jsonl'

# Prepare the data
data = []
for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            data.append({
                "file_name": class_name + "/" + img_name,
                "label": class_name
            })

# Write to metadata.jsonl
with open(output_file, 'w') as f:
    for entry in data:
        f.write(json.dumps(entry) + '\n')

print(f"Metadata file created at {output_file}")

In [ ]:
import os

# Define the base directory
base_dir = '/content/drive/MyDrive/pathmnist/images'

# Initialize total image count
total_image_count = 0

# Iterate through sub-folders and count images
for sub_folder in os.listdir(base_dir):
    sub_folder_path = os.path.join(base_dir, sub_folder)

    # Check if it's a directory
    if os.path.isdir(sub_folder_path):
        # Count image files in the sub-folder
        image_count = len([f for f in os.listdir(sub_folder_path) if os.path.isfile(os.path.join(sub_folder_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        print(f"Sub-folder: {sub_folder}, Image Count: {image_count}")

        # Add to total count
        total_image_count += image_count

# Print the total count
print(f"\nTotal Image Count: {total_image_count}")

In [ ]:
import json
from collections import defaultdict

# Define the path to the metadata file
metadata_file = '/content/drive/MyDrive/pathmnist/images/metadata.jsonl'

# Create a dictionary to store label counts
label_counts = defaultdict(int)

# Initialize total count
total_count = 0

# Open the metadata file and process each line
with open(metadata_file, 'r') as f:
    for line in f:
        # Load the JSON data from the line
        data = json.loads(line)

        # Get the label from the data
        label = data['label']

        # Increment the count for the label
        label_counts[label] += 1

        # Increment the total count
        total_count += 1

# Print the label counts
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")

# Print the total count
print(f"\nTotal Count: {total_count}")